# Calculate Straight-Line Distance for School

In [1]:
import requests
import pandas as pd
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time

In [2]:
school = pd.read_csv("../data/landing/School Locations 2023.csv")
school.head()

,Education_Sector,Entity_Type,School_No,School_Name,School_Type,School_Status,Address_Line_1,Address_Line_2,Address_Town,Address_State,...,Postal_Address_Line_1,Postal_Address_Line_2,Postal_Town,Postal_State,Postal_Postcode,Full_Phone_No,LGA_ID,LGA_Name,X,Y
0,Government,1,1,Alberton Primary School,Primary,O,21 Thomson Street,NaN,Alberton,VIC,...,21 Thomson Street,NaN,ALBERTON,VIC,3971,03 5183 2412,681,Wellington (S),146.66660,-38.61771
1,Government,1,3,Allansford and District Primary School,Primary,O,Frank Street,NaN,Allansford,VIC,...,Frank Street,NaN,ALLANSFORD,VIC,3277,03 5565 1382,673,Warrnambool (C),142.59039,-38.38628
2,Government,1,4,Avoca Primary School,Primary,O,118 Barnett Street,NaN,Avoca,VIC,...,P O Box 12,NaN,AVOCA,VIC,3467,03 5465 3176,599,Pyrenees (S),143.47565,-37.08450
3,Government,1,8,Avenel Primary School,Primary,O,40 Anderson Street,NaN,Avenel,VIC,...,40 Anderson Street,NaN,AVENEL,VIC,3664,03 5796 2264,643,Strathbogie (S),145.23472,-36.90137
4,Government,1,12,Warrandyte Primary School,Primary,O,5-11 Forbes Street,NaN,Warrandyte,VIC,...,5-11 Forbes Street,NaN,WARRANDYTE,VIC,3113,03 9844 3537,421,Manningham (C),145.21398,-37.74268


In [3]:
def distance(lat1, lon1, lat2, lon2):

    """
    Calculate the approximate distance between two sets of latitude and longitude coordinates using a simplified method.

    Parameters:
    - lat1, lon1: Latitude and longitude of the first location.
    - lat2, lon2: Latitude and longitude of the second location.

    Returns:
    - float: The approximate distance between the two locations in kilometers.
    """
    
    lat_diff = (lat2 - lat1) * 111

    # 1 degree of longitude varies, we use an approximation based on the average latitude
    avg_lat = np.radians((lat1 + lat2) / 2.0)  # Convert average latitude to radians
    lon_diff = (lon2 - lon1) * 111 * np.cos(avg_lat)

    # Calculate the distance using the Pythagorean theorem on the lat_diff and lon_diff
    return np.sqrt(lat_diff**2 + lon_diff**2)


In [4]:
def find_nearest_school(house_lat, house_lon, school):
    """
    Find the nearest distance from a given house's latitude and longitude coordinates to nearby schools.

    Parameters:
    - house_lat: Latitude of the house.
    - house_lon: Longitude of the house.
    - school: DataFrame containing school locations with 'Y' and 'X' columns.

    Returns:
    - float: The minimum distance in kilometers to the nearest school.
    """

    # Calculate the distance from the property to each school coordinate
    distances = school.apply(
        lambda row: distance(house_lat, house_lon, row['Y'], row['X']), 
        axis=1
    )

    # Return the closest distance
    return distances.min()


In [5]:
df = pd.read_csv('../data/raw/domain_outliers_removed.csv')

In [6]:
# Create another column which represents the distance to the nearest school
df['NearestSchoolDistance'] = df.apply(
    lambda row: find_nearest_school(row['Latitude'], row['Longitude'], school), 
    axis=1
)

# Output the csv file
df.to_csv("../data/raw/merge_requirement/Direct_school.csv")